## Installing Required Libraries

In [ ]:
!pip install -q transformers
!pip install -q openai-whisper
!pip install -q torchaudio
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s 

## Importing Libraries

In [ ]:
import torch
import whisper
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import numpy as np
from google.colab import files
import IPython.display as ipd

## Uploading Audio File

In [ ]:
print("Upload an audio file (wav/mp3/m4a)...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
ipd.Audio(filename)

Upload an audio file (wav/mp3/m4a)...


Saving tts_output.mp3 to tts_output (1).mp3


## Transcribing the Audio

In [ ]:
print("\nTranscribing with Whisper...")
model_whisper = whisper.load_model("base")
result = model_whisper.transcribe(filename)
transcribed_text = result["text"]
print(f"\n📝 Transcribed Text: \"{transcribed_text.strip()}\"")


Transcribing with Whisper...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



📝 Transcribed Text: "I am a very good morning."


## Load Grammar Model

In [ ]:
print("\nLoading grammar model...")
tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-CoLA")
grammar_model = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")


Loading grammar model...


Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Evaluate Transcription

In [ ]:
inputs = tokenizer(transcribed_text, return_tensors="pt", truncation=True)
outputs = grammar_model(**inputs)
probs = softmax(outputs.logits, dim=1)

acceptability_prob = probs[0][1].item()
grammar_score = round(acceptability_prob * 5, 2)

print(f"\n✅ Grammar Acceptability Score: {grammar_score} / 5")


✅ Grammar Acceptability Score: 0.27 / 5


## Convert Text to Speech and Play (To provide random audio file input)

In [ ]:
!pip install -q gTTS

from gtts import gTTS
from IPython.display import Audio, FileLink

text = "I am a very good morning"

tts = gTTS(text)
audio_file = "tts_output.mp3"
tts.save(audio_file)

display(Audio(audio_file))

display(FileLink(audio_file))


/content/tts_output.mp3